In [ ]:
import os, sys 
from math import ceil # Round up
import numpy as np 
import meshplot as mp # Visualize 3D meshes
import ipywidgets # Interactive sliders/widgets
from skimage import measure #surface extraction (marching cubes)
from scipy.ndimage import zoom # Resize
from scipy.interpolate import interpn #interpolate a volume at given points
from IPython.display import display  #For display 
from einops import rearrange # Elegant tensor reordering
import trimesh

# Utilities function

In [ ]:
# Dot product on the first dimension of n-dimensional arrays x and y
def dot(x, y):
    return np.einsum('i..., i... -> ...', x, y)

#SDF for Torus (formula: https://iquilezles.org/articles/distfunctions/)
#x[[0, 2]]: Get only XZ dimensions → projects onto XZ plane, centered at the origin
#use np.linalg.norm to computes the distance from the Y-axis
#minus the radius and thickness to calculate the SD 
#Stack with x[1]: Y component preserved → represents distance from torus tube.
def sdf_torus(x, radius, thickness):
    q = np.stack([np.linalg.norm(x[[0, 2]], axis=0) - radius, x[1]])
    return np.linalg.norm(q, axis=0) - thickness

# Crop an n-dimensional image with a centered cropping region (after zoom)
def center_crop(img, shape):
    start = [a // 2 - da // 2 for a, da in zip(img.shape, shape)] #computes center offset by minusing da (desired a)
    end = [a + b for a, b in zip(start, shape)] #start + desired shape
    slices = tuple([slice(a, b) for a, b in zip(start, end)]) #returns cropping slices for each axis
    return img[slices]

# Add noise to coordinates
def gradient_noise(x, scale, strength, seed=None):
    shape = [ceil(s / scale) for s in x.shape[1:]]
    if seed:
        np.random.seed(seed)
    scalar_noise = np.random.randn(*shape)
    scalar_noise = zoom(scalar_noise, zoom=scale)
    scalar_noise = center_crop(scalar_noise, shape=x.shape[1:])
    vector_noise = np.stack(np.gradient(scalar_noise))
    return vector_noise * strength

# Meshplot will left an annoying print statement in their code
# This function used to supress it.
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Interactive with the bump

In [ ]:
plot = None

# Makes sliders for parameters
@mp.interact(
    radius=(0, 0.5, 0.05),             #Main radius: distance from (0,0,0) to the center of the tube
    thickness=(0.01, 0.25, 0.05),      #Minor radius: distance from the center of the tube to outside
    noise_scale=(0.0, 40, 1),          
    noise_strength=(0.0, 40, 1),
    seed=(1, 100),
    bump_angle=(-1., 1., 0.1),         #Multiply with pi later to move the bump around the torus 
    bump_width=(0.001, 0.02, 0.001),   #controls the spread of the gaussian bump 
    bump_height=(0.1, 50.),            #controls the amplification
)

def show(radius, thickness, noise_scale, noise_strength, seed, bump_angle, bump_width, bump_height):
    global plot

    #Create an array of 100 points from -1 to 1 
    coords = np.linspace(-1, 1, 100) 

    #Create a 3D grid with coords above 
    x = np.stack(np.meshgrid(coords, coords, coords)) # x.shape = (3, 100, 100, 100)

    #Base torus shape (no bump)
    sdf = sdf_torus(x, radius, thickness)  #Get SDF 
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0) #Convert SDF to a mesh 

    # Noise field
    x_warp = gradient_noise(x, noise_scale, noise_strength, seed)

    # Bump field (Gaussian bump)
    angle = np.pi * bump_angle
    gaussian_center = np.array([np.sin(angle), 0., np.cos(angle)]) * radius
    x_dist = np.linalg.norm((x - gaussian_center[:, None, None, None]), axis=0)
    x_bump = bump_height * np.exp(-1. / bump_width * x_dist**2)

    # Add bump gradient to warp field
    # Gradient of bump field:  a 3D vector pointing outward from the bum
    x_warp += -np.stack(np.gradient(x_bump))

    # Interpolate displacement at vertices
    x_warp = rearrange(x_warp, 'v h w d -> h w d v')
    vertex_noise = interpn([np.arange(100)] * 3, x_warp, verts, bounds_error=False, fill_value=0)
    vertex_noise = np.nan_to_num(vertex_noise)

    # Apply deformation
    warped_verts = verts + vertex_noise

    # Visualize
    if plot is None:
        plot = mp.plot(warped_verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=warped_verts, faces=faces)
        display(plot._renderer)

    print(f"#Verts: {len(warped_verts)}, #Faces: {len(faces)}")


interactive(children=(FloatSlider(value=0.25, description='radius', max=0.5, step=0.05), FloatSlider(value=0.1…

# Export ply files (currently fix every configuration for the angle)

In [ ]:
from math import pi

#PLEASE CHANGE TO YOUR DIR, create your directory first 
out_dir = os.path.expanduser("~/Downloads/fixed_torus_biggerbump2_1404") 
os.makedirs(out_dir, exist_ok=True)

# Fixed Parameters
resolution = 100
radius = 0.25
thickness = 0.1
noise_scale = 20
noise_strength = 5
seed = 42
bump_width = 0.01
bump_height = 30.0


coords = np.linspace(-1, 1, resolution)
x = np.stack(np.meshgrid(coords, coords, coords))

for i, bump_angle in enumerate(np.linspace(-1.0, 1.0, 500)):
    # Base torus SDF
    sdf = sdf_torus(x, radius, thickness)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)

    # Noise field
    x_warp = gradient_noise(x, noise_scale, noise_strength, seed)

    # Bump field
    angle = pi * bump_angle
    gaussian_center = np.array([np.sin(angle), 0., np.cos(angle)]) * radius
    x_dist = np.linalg.norm((x - gaussian_center[:, None, None, None]), axis=0)
    x_bump = bump_height * np.exp(-1. / bump_width * x_dist**2)
    x_warp += -np.stack(np.gradient(x_bump))

    # Interpolate and deform
    x_warp = rearrange(x_warp, 'v h w d -> h w d v')
    vertex_noise = interpn([np.arange(resolution)] * 3, x_warp, verts, bounds_error=False, fill_value=0)
    vertex_noise = np.nan_to_num(vertex_noise)
    warped_verts = verts + vertex_noise

    # Save as PLY
    mesh = trimesh.Trimesh(vertices=warped_verts, faces=faces, process=False)
    mesh.export(os.path.join(out_dir, f"torus_{i:03d}.ply"))

print("Done exporting files")

NameError: name 'pi' is not defined